## Polars home page (https://pola.rs/) 
## The User Guide home page (https://docs.pola.rs/)

#Installation

pip install polars

In [1]:
from hh_transmission_model import run_SEIR_model_pl, plot_SEIR_pl, Params
from pandas_transmission_model import run_SEIR_model_pd, plot_SEIR_pd
import time

In [11]:
# Parameters
params = Params(
    no_runs=10,
    pop_size=5000,
    inf_duration=10, 
    exposed_duration=4,
    random_seed=5,
    transmission_rate=0.02,
    time_horizon=50,
    time_step=0.5,
    record_transmission=True,
    record_all_new_cases=True
)

In [12]:
pop_sizes = [500]#, 5000, 50000] #, 500000]
for pop_size in pop_sizes:
    setattr(params, 'pop_size', pop_size)  
    # get the start time
    st = time.time()
    results_pl = run_SEIR_model_pl(params)
    # get the end time
    et = time.time()
    # get the execution time
    elapsed_time_pl = et - st
    print('Pop size', pop_size,' - Polars execution time:', round(elapsed_time_pl, 1), 'seconds')   

    # get the start time
    st = time.time()
    results_pd = run_SEIR_model_pd(params)
    # get the end time
    et = time.time()
    # get the execution time
    elapsed_time_pd = et - st
    print('Pop size', pop_size,' - Pandas execution time:', round(elapsed_time_pd, 1), 'seconds')   

"""
Pop size 500  - Polars execution time: 1.5 seconds
Pop size 500  - Pandas execution time: 1.4 seconds
Pop size 5000  - Polars execution time: 2.0 seconds
Pop size 5000  - Pandas execution time: 3.8 seconds
Pop size 50000  - Polars execution time: 4.4 seconds
Pop size 50000  - Pandas execution time: 27.8 seconds
Pop size 500000  - Polars execution time: 24.3 seconds
Pop size 500000  - Pandas execution time: 273.0 seconds
"""    

Pop size 500  - Polars execution time: 0.8 seconds
Pop size 500  - Pandas execution time: 0.8 seconds


'\nPop size 500  - Polars execution time: 1.5 seconds\nPop size 500  - Pandas execution time: 1.4 seconds\nPop size 5000  - Polars execution time: 2.0 seconds\nPop size 5000  - Pandas execution time: 3.8 seconds\nPop size 50000  - Polars execution time: 4.4 seconds\nPop size 50000  - Pandas execution time: 27.8 seconds\nPop size 500000  - Polars execution time: 24.3 seconds\nPop size 500000  - Pandas execution time: 273.0 seconds\n\n'

In [14]:
#plot results
#plot_SEIR_pl(results_pl["all_transmission"])
#plot_SEIR_pd(results_pd["all_transmission"])

## Data structures in Polars

In [29]:
# Create a dataframe
import polars as pl

p = params

#via dictionary
population = pl.DataFrame(
                        {
                        "id": range(p.pop_size),
                        "state": ["Susceptible"] * p.pop_size,
                        "s_time_exposed": [0] * p.pop_size,
                        "s_time_infectious": [0] * p.pop_size,
                        "s_time_recovery": [0] * p.pop_size,
                        "exposed_from": [-1] * p.pop_size,
                        }
                        )
print(population.head())

#via list of polars Series
population2 = [
    pl.Series("id", range(p.pop_size), dtype=pl.Int64),
    pl.Series("state",  ["Susceptible"] * p.pop_size),
    pl.Series("s_time_exposed", [0] * p.pop_size, dtype=pl.Int64),
    pl.Series("s_time_infectious", [0] * p.pop_size, dtype=pl.Int64),
    pl.Series("s_time_recovery", [0] * p.pop_size, dtype=pl.Int64),
    pl.Series("exposed_from", [-1] * p.pop_size, dtype=pl.Int64),
    
]
population2 = pl.DataFrame(population2)
print(population2.head())


shape: (5, 6)
┌─────┬─────────────┬────────────────┬───────────────────┬─────────────────┬──────────────┐
│ id  ┆ state       ┆ s_time_exposed ┆ s_time_infectious ┆ s_time_recovery ┆ exposed_from │
│ --- ┆ ---         ┆ ---            ┆ ---               ┆ ---             ┆ ---          │
│ i64 ┆ str         ┆ i64            ┆ i64               ┆ i64             ┆ i64          │
╞═════╪═════════════╪════════════════╪═══════════════════╪═════════════════╪══════════════╡
│ 0   ┆ Susceptible ┆ 0              ┆ 0                 ┆ 0               ┆ -1           │
│ 1   ┆ Susceptible ┆ 0              ┆ 0                 ┆ 0               ┆ -1           │
│ 2   ┆ Susceptible ┆ 0              ┆ 0                 ┆ 0               ┆ -1           │
│ 3   ┆ Susceptible ┆ 0              ┆ 0                 ┆ 0               ┆ -1           │
│ 4   ┆ Susceptible ┆ 0              ┆ 0                 ┆ 0               ┆ -1           │
└─────┴─────────────┴────────────────┴───────────────────┴────────

In [30]:
#add rows
population3 = pl.DataFrame([
    pl.Series("id", [p.pop_size], dtype=pl.Int64),
    pl.Series("state",  ["Infectious"]),
    pl.Series("s_time_exposed", [0], dtype=pl.Int64),
    pl.Series("s_time_infectious", [0] , dtype=pl.Int64),
    pl.Series("s_time_recovery", [0] , dtype=pl.Int64),
    pl.Series("exposed_from", [-1], dtype=pl.Int64),   
])
print(population3)

# new memory slab
new_population = pl.concat([population, population3], rechunk=True)
print(new_population.tail())

# append free (no memory copy)
new_population = population.vstack(population3)
print(new_population.tail())

# try to append in place
population.extend(population3)
print(population.tail())


shape: (1, 6)
┌─────┬────────────┬────────────────┬───────────────────┬─────────────────┬──────────────┐
│ id  ┆ state      ┆ s_time_exposed ┆ s_time_infectious ┆ s_time_recovery ┆ exposed_from │
│ --- ┆ ---        ┆ ---            ┆ ---               ┆ ---             ┆ ---          │
│ i64 ┆ str        ┆ i64            ┆ i64               ┆ i64             ┆ i64          │
╞═════╪════════════╪════════════════╪═══════════════════╪═════════════════╪══════════════╡
│ 500 ┆ Infectious ┆ 0              ┆ 0                 ┆ 0               ┆ -1           │
└─────┴────────────┴────────────────┴───────────────────┴─────────────────┴──────────────┘
shape: (5, 6)
┌─────┬─────────────┬────────────────┬───────────────────┬─────────────────┬──────────────┐
│ id  ┆ state       ┆ s_time_exposed ┆ s_time_infectious ┆ s_time_recovery ┆ exposed_from │
│ --- ┆ ---         ┆ ---            ┆ ---               ┆ ---             ┆ ---          │
│ i64 ┆ str         ┆ i64            ┆ i64               ┆ 